In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi = init_hmm(T=50)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.partial(combinators.sequence, hmm_step)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi = hmm_run(50, z0, mu, sigma, pi)

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [12]:
smc_hmm = smc.smc(hmm.hmm_step, hmm.hmm_retrace)

In [13]:
samples, elbos, inference = smc.particle_mh(num_particles, hmm.init_hmm, smc_hmm, 1000, 50, smc_hmm_params, generative, use_cuda=False)

In [14]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 1.000000
SMC accuracy at time 2: 0.000000
SMC accuracy at time 3: 0.000000
SMC accuracy at time 4: 0.000000
SMC accuracy at time 5: 0.000000
SMC accuracy at time 6: 1.000000
SMC accuracy at time 7: 1.000000
SMC accuracy at time 8: 1.000000
SMC accuracy at time 9: 0.000000
SMC accuracy at time 10: 0.000000
SMC accuracy at time 11: 0.000000
SMC accuracy at time 12: 0.000000
SMC accuracy at time 13: 1.000000
SMC accuracy at time 14: 1.000000
SMC accuracy at time 15: 0.900000
SMC accuracy at time 16: 0.100000
SMC accuracy at time 17: 1.000000
SMC accuracy at time 18: 0.900000
SMC accuracy at time 19: 0.020000
SMC accuracy at time 20: 1.000000
SMC accuracy at time 21: 0.000000
SMC accuracy at time 22: 0.100000
SMC accuracy at time 23: 0.120000
SMC accuracy at time 24: 0.900000
SMC accuracy at time 25: 1.000000
SMC accuracy at time 26: 0.020000
SMC accuracy at time 27: 0.100000
SMC accuracy at time 28: 0.880000
SMC accuracy at time 29: 0.980000
SMC accuracy at time 30

In [15]:
elbos

tensor([-123.0490, -122.1571, -119.4580, -119.4580, -119.4580, -119.4580,
        -119.4580, -119.4580, -119.4580, -119.4580, -119.4580, -119.4580,
        -119.4580, -119.4580, -119.4580, -119.4580, -119.4580, -119.4580,
        -119.4580, -119.4580, -119.4580, -123.0523, -121.0510, -121.0510,
        -121.0510, -121.0510, -121.0510, -121.0510, -121.0510, -121.0510,
        -121.0510, -121.0510, -121.0510, -121.0510, -121.0510, -121.0510,
        -121.0510, -121.0510, -121.0510, -121.0510, -120.4295, -119.7206,
        -119.7206, -119.7206, -120.9925, -119.9446, -119.9446, -119.9444,
        -119.9444, -119.9444, -119.9444, -119.9444, -119.9444, -119.9444,
        -119.9444, -119.9444, -119.9444, -119.9444, -119.9444, -119.9444,
        -119.9444, -119.9444, -119.9444, -119.9444, -119.9444, -119.9444,
        -119.9444, -119.9444, -119.9444, -119.9444, -119.9444, -119.9444,
        -119.9444, -119.9444, -119.9444, -119.9444, -119.9444, -119.4134,
        -119.4134, -119.4134, -119.413